# The Parity Problem
You are given a function as a black box: $ f: \{0, 1\}^n \rightarrow \{0, 1\}$ and $f(x) = u.x \mod2$.

For some hidden $u \in \{0, 1\}^n$, find $u$ with as few queries as possible. This is a restricted version of the Deutsch–Jozsa algorithm where instead of distinguishing between two different classes of functions, it tries to learn a string encoded in a function. 

## A classical solution

![Classical Solution](bv.PNG "A classical solution the parity problem")

We need $n$ queries to compute the hidden string, because each time you run this circuit, you get only one bit of information. 
For example: let the hidden string $u = 101 $.

$$ f(100) = 1 $$ 

$$ f(010) = 0 $$

$$ f(001) = 1 $$

One possible solution to implement this classically is to use a classical AND gate as a mask to get each bit of information. 

## The Bernstein-Vazirani Algorithm
In contrast to the classical solution, the quantum solution requires only one query to find the hidden $u$. This algorithm was invented by Ethan Bernstein and Umesh Vazirani in 1992. Your task is to implement this algorithm in Q#. You can read more about the algorithm [here](https://en.wikipedia.org/wiki/Bernstein%E2%80%93Vazirani_algorithm).
![Quantum Solution](bv-q.PNG "A quantum solution the parity problem")

The Bernstein-Vazirani algorithm is described as follows:
1. Initialize n query qubits in the state $|0\rangle$ and one answer qubit in the state $|-\rangle$
2. Apply the Hadamard Transform to an n-qubit query state to get $H|0\rangle^{\otimes n} =  \frac{1}{2^{n/2}} \sum_x |x> $.
3. Apply the oracle $U_f$ which transforms the state to  $\frac{1}{2^{n/2}} \sum_x (-1)^{u.x} |x> $
4. Apply the Hadamard Transform to the query state after the oracle is applied.
5. Measure in the computational basis.

## Task 1
**Summary**

Prepare a qubit state in a uniform superposition

**Input**

*query*: an array of query qubits

**Output** 

Apply the hadamard transform to n qubits in state $|0\rangle$. 


In [72]:
import qsharp
import qsharp.azure
import matplotlib.pyplot as plt

In [73]:
%%qsharp 
open Microsoft.Quantum.Diagnostics; 
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Random;

In [74]:

%%qsharp
operation WalshHadamardTransform(query: Qubit[]): Unit is Adj{
    for q in query{
        H(q);
    }
}

## Task 2
**Summary**

Prepare the answer qubit state in the state $|-\rangle$

**Input**

*answer*: the answer qubit

In [75]:
%%qsharp
operation PrepareAnswerQubit(answer: Qubit): Unit {
    X(answer);
    H(answer);
}

## Task 3


**Summary**

Encode the secret string (supplied as a boolean array s) into a function $U_f$

**Input**: 
query: qubit array
answer: answer qubit
s: secret string as a boolean array


In [76]:
%%qsharp
operation Uf(query : Qubit[], answer: Qubit, s: Bool[]): Unit {
    for i in 0..(Length(s) - 1){
        if(s[i] == true){
            Z(query[i]);
        }
    }
}

## Task 4: Explain what gates you applied to create the encoded string as $U_f$ and why you applied them.

I'm not sure if I fully understand this algorithm yet, but what I think I did was apply a phase change to each qubit for each bit encoded as a 1 in string s.  This results in the intial $\ket{0}$ being transformed to $\ket{1}$ after the hadamard transformations.

## Task 5: Run the Bernstein-Vazirani Algorithm using operations defined above

1. Create the secret string, and the required qubits
1. Perform the Bernstein Vazirani algorithm by using the tasks above. 
1. Measure the resulting query qubits. 
1. Transform the query results into an integer array which should give you the secret string. Return this integer array.

**Notice the hadamard-oracle-hadamard structure of the circuit. You can use the within..apply pattern in Q\# to do this. (Lecture 8 programming slides, Slide\# 26 )** 

In [77]:
%%qsharp
operation BernsteinVazirani () : Int [] {
    //mutable result = [0, size=6]; //Change the size according to the size of your secret string
    mutable result = [];
    use q = Qubit[6];
    use answer = Qubit();
    let s = [true, false, false, true, false, false];
    PrepareAnswerQubit(answer);
    within{
        WalshHadamardTransform(q);
    }
    apply{
        Uf(q, answer, s);
    }
    for qubit in q{
        let measurement = Measure ([PauliZ], [qubit]) == Zero ? 0|1;
        set result += [measurement];
    }
    
    let _ = Measure([PauliZ],[answer]);
    return result;
}

In [78]:
result = BernsteinVazirani.simulate()
print(result)

[1, 0, 0, 1, 0, 0]
